In [1]:
# ! pip3 install --upgrade --user google-cloud-aiplatform

import vertexai
from vertexai.generative_models import GenerativeModel
from google.cloud import storage
import io,json 

from PIL import Image as PIL_Image

# Initialize Vertex AI
PROJECT_ID = ""
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

bucket_name = ""
multimodal_model = GenerativeModel("gemini-1.0-pro-vision")

def load_image_from_gcs(bucket_name: str, blob_name: str) -> 'Image':
    """Load an image from Google Cloud Storage into the format expected by Vertex AI."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    image_bytes = blob.download_as_bytes()
    
    pil_image = PIL_Image.open(io.BytesIO(image_bytes))
    pil_image_rgb = pil_image.convert("RGB")
    
    img_byte_arr = io.BytesIO()
    pil_image_rgb.save(img_byte_arr, format='JPEG')
    img_byte_arr = img_byte_arr.getvalue()
    
    return vertexai.generative_models.Image.from_bytes(img_byte_arr)

In [2]:
sneaker_image_paths = [
    "sneaker-collections/nike1.jpg",
    "sneaker-collections/nike2.jpg",
    "sneaker-collections/nike3.jpg",
    "sneaker-collections/nike4.jpg",
    "sneaker-collections/nike5.jpg",
    "sneaker-collections/adidas1.png",
    "sneaker-collections/adidas2.png",
    "sneaker-collections/adidas3.png",
    "sneaker-collections/adidas4.png",
    "sneaker-collections/adidas5.png"   
]
sneaker_images = [load_image_from_gcs(bucket_name, path) for path in sneaker_image_paths]

In [3]:
input_image_path = "user-input-images/input2.png"
input_image = load_image_from_gcs(bucket_name, input_image_path)

In [4]:
prompt = "Describe just the tshirt,shirt,trousers or jeans in the picture in brief"
contents = [
    prompt,
    input_image,
]
responses = multimodal_model.generate_content(contents, stream=True)

for response in responses:
    print(response.text)

 The image shows a pair of blue jeans. The jeans are straight-legged and have a medium rise. They are made from a cotton blend fabric and have
 a zip fly and button closure.


In [5]:
fashion_expert_prompt = """You are a fashion expert. 
                    Recommend 2 adidas sneakers which would go along with the input image
                    along with the price and also explain why is it a good match given the input picture specifically. 
                    Respond in json with 2 keys explanation and image_path.
                    Output Format must be strictly a json string in the below format with no illegal string terminations
                    and must be inside quotes.
                    "{'explanation':'','image_paths':''}"
                    """
contents = [
    "Consider the following sneakers:",
    "Nike Air Max 1:",
    sneaker_images[0],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike1.jpg?authuser=4",
    "Nike Air max SC:",
    sneaker_images[1],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike2.jpg?authuser=4",
    "Nike Air Max 1 Black:",
    sneaker_images[2],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike3.jpg?authuser=4",
    "Nike Dunk Low Retro- White:",
    sneaker_images[3],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike4.jpg?authuser=4",
    "Nike Dunk Low Retro- Black:",
    sneaker_images[4],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/nike5.jpg?authuser=4",
    "Adidas-Sambda OG Shoes:",
    sneaker_images[5],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas1.png?authuser=4",
    "Adidas-Rivalry Hi shoes:",
    sneaker_images[6],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas2.png?authuser=4",
    "Adidas-Ultrabounce Shoes:",
    sneaker_images[7],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas3.png?authuser=4",
    "Adidas-Courtblock shoes:",
    sneaker_images[8],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas4.png?authuser=4",
    "Adidas-Stan Smith shoes:",
    sneaker_images[9],"https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas5.png?authuser=4",
    "Input Image:",
    input_image,
    fashion_expert_prompt,
]
responses = multimodal_model.generate_content(contents, stream=True)

final_output = ""
for response in responses:
    llm_output= response.text
    clean_json_string = llm_output.strip()
    cleaned_output = clean_json_string.strip("`").replace("json\n", "", 1).strip()
    # clean_json_string = ' '.join(clean_json_string.split())
    # llm_output_dict = json.loads(clean_json_string)
    final_output += cleaned_output
    # final_output = json.loads(final_output)

print(final_output)

{
                    "explanation": "The Adidas Stan Smith shoes are a perfect match for the input image. The shoes are a classic white sneakerwith a blue heel tab and blue stripes. The jeans in the input image are a medium blue wash with a straight leg. The shoes and jeans are both casualand stylish, and they would look great together. \n\n The Adidas Ultraboost shoes are also a good match for the input image. The shoes area black running shoe with a blue boost sole. The jeans in the input image are a medium blue wash with a straight leg. The shoes and jeans are bothathletic and stylish, and they would look great together.",
                    "image_paths": ["https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas5.png?authuser=4", "https://storage.cloud.google.com/llm-demo-v1/sneaker-collections/adidas3.png?authuser=4"]
                    }


In [1]:
print(json.loads(final_output))

NameError: name 'json' is not defined